This idea of this notebook is to create a first (simple) DL model using all the same features used in the multinomial logit model.

Much inspiration was derived from https://towardsdatascience.com/use-machine-learning-to-predict-horse-racing-4f1111fb6ced.

In [175]:
import math
from importlib import reload
import deeplearninglib
reload(deeplearninglib)
from deeplearninglib import *

In [176]:
# read in data

# select the same features used in the multinomial model
X_columns = ["race_id", "stall_number", "age", "sire_sr", "dam_sr", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "visor", "cheekpieces", "tonguetie"]
y_columns = ["win"]

train_data = RacesDataset('data\\runners_train.csv', X_columns, y_columns)
test_data = RacesDataset('data\\runners_test.csv', X_columns, y_columns)

train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [177]:
train_data.races

age                                                           \
stall_number   1     2    3    4    5    6    7    8     9    10   11   12   
race_id                                                                      
11504         3.0   3.0  3.0  3.0  3.0  3.0  3.0  3.0   0.0  0.0  0.0  0.0   
11505         4.0   6.0  9.0  7.0  5.0  8.0  5.0  5.0   5.0  4.0  5.0  0.0   
11506         3.0   3.0  3.0  3.0  3.0  3.0  3.0  3.0   3.0  3.0  0.0  0.0   
11507         5.0   4.0  4.0  4.0  4.0  4.0  8.0  4.0   4.0  0.0  0.0  0.0   
11508         7.0   4.0  5.0  4.0  5.0  4.0  5.0  5.0   4.0  0.0  0.0  0.0   
...           ...   ...  ...  ...  ...  ...  ...  ...   ...  ...  ...  ...   
440435        4.0   9.0  5.0  4.0  8.0  6.0  4.0  7.0   4.0  4.0  0.0  0.0   
440436        8.0   4.0  9.0  5.0  4.0  5.0  7.0  6.0   4.0  0.0  0.0  0.0   
440630        2.0   2.0  2.0  2.0  2.0  2.0  0.0  0.0   0.0  0.0  0.0  0.0   
440730        5.0   5.0  5.0  7.0  6.0  6.0  8.0  3.0  11.0  5.0  5.0  6.0   
440873        5.0  11.0  6.0  7.0  5.0  7.0  5.0  4.0   9.0  0.0  0.0  0.0   

                                 blinkers                                     \
stall_number   13   14   15   16       1    2    3    4    5    6    7    8    
race_id                                                                        
11504         0.0  0.0  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11505         0.0  0.0  0.0  0.0      0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
11506         0.0  0.0  0.0  0.0      1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0   
11507         0.0  0.0  0.0  0.0      0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
11508         0.0  0.0  0.0  0.0      1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...           ...  ...  ...  ...      ...  ...  ...  ...  ...  ...  ...  ...   
440435        0.0  0.0  0.0  0.0      0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0   
440436        0.0  0.0  0.0  0.0      1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
440630        0.0  0.0  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
440730        5.0  0.0  0.0  0.0      1.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0   
440873        0.0  0.0  0.0  0.0      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                                     cheekpieces            \
stall_number   9    10   11   12   13   14   15   16          1    2    3    
race_id                                                                      
11504         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0   
11505         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0   
11506         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0   
11507         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  1.0   
11508         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  1.0  1.0   
...           ...  ...  ...  ...  ...  ...  ...  ...         ...  ...  ...   
440435        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0   
440436        1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0   
440630        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0   
440730        0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0   
440873        1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0   

                                                                               \
stall_number   4    5    6    7    8    9    10   11   12   13   14   15   16   
race_id                                                                         
11504         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11505         0.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0   
11506         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11507         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11508         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...           ...  ...  ...  ...  ...  ...  ...  ...  ...  

In [178]:
pd.options.display.max_columns = 1000 # was 20
train_data.races.iloc[:, train_data.X_columns].head()

age                                                              \
stall_number   1    2    3    4    5    6    7    8    9    10   11   12   13   
race_id                                                                         
11504         3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  0.0  0.0  0.0  0.0  0.0   
11505         4.0  6.0  9.0  7.0  5.0  8.0  5.0  5.0  5.0  4.0  5.0  0.0  0.0   
11506         3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  0.0  0.0  0.0   
11507         5.0  4.0  4.0  4.0  4.0  4.0  8.0  4.0  4.0  0.0  0.0  0.0  0.0   
11508         7.0  4.0  5.0  4.0  5.0  4.0  5.0  5.0  4.0  0.0  0.0  0.0  0.0   

                            blinkers                                          \
stall_number   14   15   16       1    2    3    4    5    6    7    8    9    
race_id                                                                        
11504         0.0  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11505         0.0  0.0  0.0      0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0   
11506         0.0  0.0  0.0      1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0   
11507         0.0  0.0  0.0      0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0   
11508         0.0  0.0  0.0      1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                                cheekpieces                 \
stall_number   10   11   12   13   14   15   16          1    2    3    4    
race_id                                                                      
11504         0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0  0.0   
11505         0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0  0.0   
11506         0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0  0.0   
11507         0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  1.0  0.0   
11508         0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  1.0  1.0  0.0   

                                                                          \
stall_number   5    6    7    8    9    10   11   12   13   14   15   16   
race_id                                                                    
11504         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11505         0.0  0.0  1.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0   
11506         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11507         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11508         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

             dam_sr                                                         \
stall_number     1    2    3    4    5    6    7    8    9    10   11   12   
race_id                                                                      
11504           0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11505           0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11506           0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11507           0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11508           0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                 daysLTO                                     \
stall_number   13   14   15   16      1      2      3      4      5      6    
race_id                                                                       
11504         0.0  0.0  0.0  0.0   365.0  365.0  365.0  365.0  365.0  365.0   
11505         0.0  0.0  0.0  0.0   365.0  365.0  365.0  365.0  365.0  365.0   
11506         0.0  0.0  0.0  0.0   365.0  365.0  365.0  365.0  365.0  365.0   
11507         0.0  0.0  0.0  0.0   365.0  365.0  365.0  365.0  365.0  365.0   
11508         0.0  0.0  0.0  0.0   365.0  365.0  365.0  365.0  365.0  365.0   

                                                                          \
stall_number     7      8      9      10     11   12   13   14   15   16   
race_id                                                                    
1150

In [179]:
# build the neural network

output_layer_nodes = train_data.y.shape[1]
input_layer_nodes = train_data.X.shape[1]

# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
print(f"Using {device} device")

model = LRLS(input_layer_nodes, output_layer_nodes).to(device) # linear-relu-linear-softwax neural net with 1 hidden layer
print(model)

print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Using cpu device
LRLS(
  (neural_network): Sequential(
    (0): Linear(in_features=368, out_features=77, bias=False)
    (1): ReLU()
    (2): Linear(in_features=77, out_features=16, bias=False)
    (3): Softmax(dim=1)
  )
)
Model structure: LRLS(
  (neural_network): Sequential(
    (0): Linear(in_features=368, out_features=77, bias=False)
    (1): ReLU()
    (2): Linear(in_features=77, out_features=16, bias=False)
    (3): Softmax(dim=1)
  )
)


Layer: neural_network.0.weight | Size: torch.Size([77, 368]) | Values : tensor([[ 2.0986e-02,  4.0846e-02,  5.8309e-03, -1.6257e-02,  5.2026e-02,
         -4.1414e-02, -4.2926e-02,  7.5830e-03, -5.5669e-03, -1.4908e-02,
         -4.6399e-02,  1.4583e-02,  2.1440e-02,  4.9157e-02,  2.2291e-02,
          2.2515e-03, -3.6423e-02,  2.5963e-02, -1.9848e-02,  1.8430e-02,
          4.4028e-03,  2.2929e-02,  2.0725e-02,  3.0983e-02,  5.0925e-02,
         -6.2710e-03,  1.2881e-02, -2.9378e-02,  3.5004e-02,  4.3188e-03,
         -3.2658e-03,  5.0775e-02,

In [180]:
# example to show how model is used from prediction

X = torch.rand(1, input_layer_nodes, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")
print(X.shape)

Predicted class: tensor([12])
torch.Size([1, 368])


In [181]:
# optimizing model parameters

# hyperparameters
learning_rate = 1e-3
epochs = 10

# initialize the loss function
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.697968  [   64/16620]
loss: 2.751581  [ 6464/16620]
loss: 2.751438  [12864/16620]
Test Error: 
 Accuracy: 11.8%, Avg loss: 2.755381 

Epoch 2
-------------------------------
loss: 2.735773  [   64/16620]
loss: 2.744577  [ 6464/16620]
loss: 2.746519  [12864/16620]
Test Error: 
 Accuracy: 12.6%, Avg loss: 2.750058 

Epoch 3
-------------------------------
loss: 2.729182  [   64/16620]
loss: 2.734929  [ 6464/16620]
loss: 2.731366  [12864/16620]
Test Error: 
 Accuracy: 12.7%, Avg loss: 2.747999 

Epoch 4
-------------------------------
loss: 2.726919  [   64/16620]
loss: 2.717818  [ 6464/16620]
loss: 2.728962  [12864/16620]
Test Error: 
 Accuracy: 12.5%, Avg loss: 2.747116 

Epoch 5
-------------------------------
loss: 2.722589  [   64/16620]
loss: 2.702988  [ 6464/16620]
loss: 2.725893  [12864/16620]
Test Error: 
 Accuracy: 12.5%, Avg loss: 2.746388 

Epoch 6
-------------------------------
loss: 2.716872  [   64/16620]
loss: 2.695474  [ 64